In [1]:

using StaticArrays
using AeroMDAO
using Optim

┌ Info: Precompiling AeroMDAO [ae1513eb-aba2-415b-9b88-80f66c7c7c76]
└ @ Base loading.jl:1317
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for

In [2]:
# Helper functions
#============================================#

function run_case(wing, V, α, ρ, span_num, chord_num)
    uniform = Freestream(V, α, 0.0, zeros(3))
    coeffs  = solve_case(wing, uniform, 
                         rho_ref = ρ,
                         span_num = span_num,
                         chord_num = chord_num,
                         viscous = true)[2]
end

# Objective function
evaluate_CDi(wing, V, α, ρ, span_num, chord_num) = run_case(wing, V, α, ρ, span_num, chord_num)[1]

# Lift and area constraint function
function evaluate_cons(wing, V, α, ρ, span_num, chord_num)
    area = projected_area(wing)
    ff   = run_case(wing, V, α, ρ, span_num, chord_num)
    lift = dynamic_pressure(ρ, V) * S * ff[5]
    CDi  = ff[1]

    lift, area, CDi
end

evaluate_cons (generic function with 1 method)

Optimization

In [3]:
# Parameters
V, α, ρ  = 29., 5., 1.225

(29.0, 5.0, 1.225)

In [28]:
# Design variables
n    = 8
wing = Wing(foils     = fill(naca4((2,4,1,2)), n),
            chords    = fill(0.314, n),
            twists    = fill(0.0, n),
            spans     = fill(1.3/(n-1), n-1),
            dihedrals = fill(0., n-1),
            sweeps      = fill(0., n-1))

x0 = [(chords ∘ right)(wing); α]

make_wing(x) = Wing(chords    = x, 
                    foils     = foils(right(wing)),
                    spans     = spans(right(wing)),
                    twists    = rad2deg.(twists(right(wing))), 
                    dihedrals = rad2deg.(dihedrals(right(wing))),
                    sweeps      = rad2deg.(sweeps(right(wing))))

make_wing (generic function with 1 method)

In [29]:
# Meshing and assembly
wing_mac = mean_aerodynamic_center(wing);
b, S, c  = info(wing)[1:end-1]

span_num  = 12
chord_num = 5

# Test
test_CDi = @time evaluate_CDi(wing, V, α, ρ, span_num, chord_num)
test_con = @time evaluate_cons(wing, V, α, ρ, span_num, chord_num)

# Bounds and constraints
weight      = 12 * 9.81
load_factor = 1.5
lift_req    = weight * load_factor

l_bound = fill(1e-12, length(x0))
u_bound = fill(Inf,   length(x0))

lc = [ lift_req, projected_area(wing), 0. ]
uc = [ lift_req, projected_area(wing), Inf ]

  0.013070 seconds (95.58 k allocations: 6.276 MiB)
  0.011929 seconds (95.59 k allocations: 6.277 MiB)


3-element Vector{Float64}:
 176.57999999999998
   0.8164
  Inf

In [31]:
# Closures
evaluate_CDi(x) = evaluate_CDi(make_wing(x[1:end-1]), V, x[end], ρ, span_num, chord_num)
cons!(cs, x) = cs .= evaluate_cons(make_wing(x[1:end-1]), V, x[end], ρ, span_num, chord_num)

cons! (generic function with 1 method)

In [24]:
## Optim variables
optimize_chords = TwiceDifferentiable(evaluate_CDi, x0)
cons_lift_area  = TwiceDifferentiableConstraints(cons!, l_bound, u_bound, lc, uc);

In [25]:
## Run optimisation
res_func = optimize(optimize_chords,      # Objective functions
                    cons_lift_area,       # Constraint
                    x0,                   # Initial value
                    IPNewton(),           # Optimization algorithm
                    autodiff = :forward,  # Automatic differentiation
                    Optim.Options(
                                #   extended_trace = true,
                                  show_trace = true
                                 )
                   )

Iter     Lagrangian value Function value   Gradient norm    |==constr.|      μ
     0   5.928190e+08     1.731895e-02     6.267752e+08     5.928190e+08     8.11e-06
 * time: 1.629871091262e9
     1   3.291749e+08     1.386798e-02     5.549290e+08     3.291749e+08     8.11e-07
 * time: 1.629871095499e9
     2   1.073264e+07     1.120641e-02     4.701800e+08     1.073264e+07     6.43e-07
 * time: 1.629871099581e9
     3   -8.251434e+06    9.970550e-03     1.909068e+08     8.251434e+06     2.59e-07
 * time: 1.629871103592e9
     4   -4.578108e+07    9.120602e-03     1.885774e+08     4.578108e+07     4.50e-07
 * time: 1.629871107617e9
     5   -3.744360e+07    8.977372e-03     1.496423e+08     3.744360e+07     1.08e-06
 * time: 1.629871111501e9
     6   9.811596e-03     8.803764e-03     1.142857e+01     9.968558e-04     1.14e-06
 * time: 1.629871115307e9
     7   9.652226e-03     9.155636e-03     6.205454e+00     4.927203e-04     4.04e-07
 * time: 1.629871119612e9
     8   9.599100e-03    

 * Status: success

 * Candidate solution
    Final objective value:     8.418866e-03

 * Found with
    Algorithm:     Interior Point Newton

 * Convergence measures
    |x - x'|               = 5.33e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.43e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 4.72e-10 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.61e-08 ≰ 0.0e+00
    |g(x)|                 = 1.31e-02 ≰ 1.0e-08

 * Work counters
    Seconds run:   215  (vs limit Inf)
    Iterations:    51
    f(x) calls:    105
    ∇f(x) calls:   105


In [26]:
## Sanity check
x_opt             = res_func.minimizer
CDi_opt, cons_opt = evaluate_CDi(x_opt), cons!(zeros(3), x_opt)

(0.008418866049837203, [176.58000000000115, 0.8164, 0.008418866049837203])

In [27]:
## Plotting
#============================================#

using Plots
plotlyjs(dpi = 300)

ini_plan = plot_planform(wing)

opt_wing = make_wing(x_opt[1:end-1])
opt_plan = plot_planform(opt_wing)

b = span(opt_wing)
plot(xlim = (0, b), zlim = (-b/2, b/2))
plot!(ini_plan, label = "Initial")
plot!(opt_plan, color = :blue, label = "Optimized")